<a href="https://colab.research.google.com/github/pranayprasad3/BERT/blob/main/DisitilBERT_For_Sequence_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sequence Classification with IMDb Reviews

We will download, tokenize, and train a model on the IMDb reviews dataset. 

## Downloading the dataset

In [1]:
import os
import tarfile
from torchvision.datasets.utils import download_url

In [2]:
dataset_url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
download_url(dataset_url, '.')

In [3]:
with tarfile.open('./aclImdb_v1.tar.gz', 'r:gz') as tar:
    tar.extractall(path='/content/')

## Exploring the folder structure

We can see that the data is arranged in test and train folders. The files 'README', 'imdb.vocab', 'imdbEr.txt' are metadata files and are not important in this context.

In [4]:
data_dir = '/content/aclImdb'

print(os.listdir(data_dir))

['imdbEr.txt', 'train', 'test', 'imdb.vocab', 'README']


The data can be divided into 3 classes (Again ignore metadata files) :
1. pos - positive sentiment 2. neg - negative sentiment 3. unsup - unclear sentiment

In [5]:
classes = os.listdir(data_dir + "/train")
print(classes)

['unsupBow.feat', 'unsup', 'pos', 'urls_pos.txt', 'labeledBow.feat', 'neg', 'urls_neg.txt', 'urls_unsup.txt']


Lets see the number of positive and negative sentiment example files we have :

In [6]:
pos_files = os.listdir(data_dir + "/train/pos")
print('No. of training examples for positivie sentiment:', len(pos_files))
print(pos_files[:5])

No. of training examples for positivie sentiment: 12500
['7253_10.txt', '7924_9.txt', '6955_10.txt', '4157_8.txt', '6734_10.txt']


In [7]:
neg_files = os.listdir(data_dir + "/train/neg")
print('No. of training examples for negative sentiment:', len(neg_files))
print(neg_files[:5])

No. of training examples for negative sentiment: 12500
['11976_1.txt', '4773_1.txt', '169_1.txt', '550_4.txt', '8000_4.txt']


So there are 12500 files each of positive and negative sentiment. Each file is scaled between 1-10 where. Note are no files with rating of 5 or 6 , this maybe because dataset creators didn't want any false positives / negatives .

## Assigning datasets for texts and it's corresponding labels.

In [8]:
from pathlib import Path

def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

In [9]:
train_texts[0]

"Not sure one can call this an anti-war film, it shows war at an elite level. These are elite troops that know what they are doing and take great pride in it. Even when they are pacifist, they still enjoy the skill level and defeating their foes, even if it does go against being a pacifist. The movies is slow and rather uneventful and in many ways is rather tame as war movies go-more so by todays standards, no body parts flying off as in modern movies. It is brutal in other ways though as you see killing at a personal level. This is more of a thinking man's movie. Once you start to watch you don't want to miss anything. The thoughts of the men in the movie and their interactions, is what the movie is about- not the combat itself or a big exciting storyline. This maybe called a war triller.<br /><br />If you are into the skill of war, if you are into reading or seeing programs about the SAS and so on, YOU WANT TO WATCH THIS MOVIE!!!!!<br /><br />Comparable movies are The Hill (1965) wit

In [10]:
train_labels[0]

1

In [11]:
train_texts[17000]

"Rather then long dance sequences and close ups of the characters which made the film drag on - the movie would have been better served explaining the story and motivations of the characters.<br /><br />The marginalisation of Nubo, the minister, auntie, mother - and the dumbing down of the dynamic and IMPORTANT rivalry between hatsumo and mameha and hatsumo and sayuri made the movie lack any real depth. If you hadn't read the book you would not really understand why Sayuri loved the Chairman and why Mameha became her mentor at all.<br /><br />Visually the film was stunning - and the actors all did the best with the C rate script they were given, but that was all that was good about this movie."

In [12]:
train_labels[17000]

0

## Splitting the training set into training and validation set

In [13]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

## Now let’s tackle tokenization. We’ll eventually train a classifier using pre-trained DistilBert, so let’s use the DistilBert tokenizer.

In [14]:
!pip install transformers
from transformers import DistilBertTokenizerFast

     |████████████████████████████████| 1.5MB 8.3MB/s 
     |████████████████████████████████| 890kB 26.2MB/s 
     |████████████████████████████████| 2.9MB 51.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=79eadde3811a0df8304db2c51925a494703b1a66afb25db9380af7f1d12a04f0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [15]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Now we can simply pass our texts to the tokenizer. We’ll pass truncation=True and padding=True, which will ensure that all of our sequences are padded to the same length and are truncated to be no longer model’s maximum input length. This will allow us to feed batches of sequences into the model at the same time.

In [16]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [17]:
train_encodings[0:5]

[Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

##  Turning our labels and encodings into a Dataset object.

In [18]:
import torch

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

## Fine-tuning with native PyTorch

In [19]:
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in tqdm(train_loader):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()
        optim.zero_grad()
model.eval()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [22]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = './model_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
#torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to ./model_save/


('./model_save/tokenizer_config.json',
 './model_save/special_tokens_map.json',
 './model_save/vocab.txt',
 './model_save/added_tokens.json')

## Validation step

In [49]:
import torch.nn.functional as F

In [27]:
val_loader = DataLoader(val_dataset, batch_size=16)

In [28]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [139]:
def validation(validation_dataloader):
  with torch.no_grad():
    loss_val_list = []
    preds_list = []
    accuracy_list = []
    accuracy_sum = 0
    for batch in tqdm(validation_dataloader):
      #print(batch.keys())
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)

      outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
      loss = outputs[0]
      logits = F.softmax(outputs[1], dim=1)   # Taking the softmax of output
      _,preds = torch.max(logits, dim=1)      # Taking the predictions of our batch
      acc = accuracy(logits,labels)           # Calculating the accuracy of current batch
      accuracy_sum += acc                     # Taking sum of all the accuracies of all the batches. This sum will be divided by batch length to get mean accuracy for validation dataset

      loss_val_list.append(loss)
      preds_list.append(preds)
      accuracy_list.append(acc)

  mean_accuracy = accuracy_sum / len(validation_dataloader)
  return mean_accuracy

In [140]:
validation(val_loader)

tensor(0.9086)

The accuracy in validation set is 91 %

## Testing our model in test dataset

In [90]:
test_loader = DataLoader(test_dataset, batch_size=16)

In [91]:
validation(test_loader)

tensor(0.9172)

The accuracy in test set is 91% ! 

## Demo

Lets take a negative review of movie Tenet

In [120]:
review = ['The movie started with a very intense "Batman" like opening. This movie showed promise the first twenty minutes and even though nothing made sense, there was still hope that it would all be tied together and all make sense. Unfortunately, hope was lost quickly and the plot disappeared into a endless past-future alternate reality abyss and never came back. This movie lacked depth and seemed pretentious from Nolan. A truly intellectual "flex" that surely he had no idea what was going on either. If there was one good thing from Covid is- very few people have had to sit through this disaster of a movie.']

Tokenising :

In [122]:
review_tokenised = tokenizer(review, truncation=True, padding=True)

Turning our labels and encodings into a Dataset object :

In [123]:
review_dataset = IMDbDataset(review_tokenised, [0])

Prediction step :

In [124]:
review_loader = DataLoader(review_dataset, batch_size=1)

In [129]:
with torch.no_grad():
  for batch in review_loader : 
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    prediction = model(input_ids, attention_mask=attention_mask, labels=labels)
    logits = F.softmax(prediction[1])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  import sys


Below we can see that class 0 has 99.66% probability therefore the prediction will be 0 (negative)

In [127]:
logits

tensor([[0.9966, 0.0034]], device='cuda:0')

The Prediction

In [133]:
_,preds = torch.max(logits, dim=1)   

In [134]:
preds.item()

0

The model correctly predicts the review to be negative.